In [ ]:
import cv2
import numpy as np
from playsound import playsound
import smtplib

Fire_Repoted = 0
Alarm_Status = False


def play_audio():
    playsound("distant-ambulance-siren-6108.mp3", True)


def send_email():
    recipient_email = "recipient_email"
    recipient_email = recipient_email.lower()

    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.ehlo()
        server.starttls()
        server.login("your_email@gmail.com", "your_email_password")
        server.sendmail('your_email@gmail.com', recipient_email, "Subject: Fire Alert\n\nA fire accident has been reported.")
        print("Email sent to {}".format(recipient_email))
        server.close()
    except Exception as e:
        print(e)


video = cv2.VideoCapture("video-fire.mp4")
# video = cv2.VideoCapture(1)

while True:
    ret, frame = video.read()
    frame = cv2.resize(frame, (960, 540))
    blur = cv2.GaussianBlur(frame, (15, 15), 0)
    hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)

    lower = [18, 50, 50]
    upper = [35, 255, 255]

    lower = np.array(lower, dtype='uint8')
    upper = np.array(upper, dtype='uint8')

    mask = cv2.inRange(hsv, lower, upper)
    output = cv2.bitwise_and(frame, hsv, mask=mask)

    size_of_fire = cv2.countNonZero(mask)

    if int(size_of_fire) > 10000:
        print("Fire Detected")
        Fire_Repoted = Fire_Repoted + 1

        if Fire_Repoted >= 1:
            if not Alarm_Status:
                send_email()
                play_audio()
                Alarm_Status = True

    if ret == False:
        break

    cv2.imshow("Output", hsv)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cv2.destroyAllWindows()
video.release()


Fire Detected
(535, b'5.7.8 Username and Password not accepted. Learn more at\n5.7.8  https://support.google.com/mail/?p=BadCredentials t23-20020a170902b21700b001d0828f2ebfsm1433211plr.273 - gsmtp')
